# Create SageMaker session and role

In [ ]:
# S3 prefix
s3_bucket = 'srt-sm'
prefix = 'training'

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# Create a SageMaker Scikit estimator 
To run our Scikit-learn training script on SageMaker, we construct a `sagemaker.sklearn.estimator.sklearn` estimator, which accepts several constructor arguments:

 - **entry_point**: The path to the Python script SageMaker runs for training and prediction.
role: Role ARN
 - **train_instance_type (optional)**: The type of SageMaker instances for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
 - **sagemaker_session (optional)**: The session used to train on Sagemaker.

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'linear_learner/sklearn_featurizer.py'

sklearn_preprocessor = SKLearn(entry_point=script_path,
                               role=role,
                               train_instance_type="ml.c4.xlarge",
                               sagemaker_session=sagemaker_session)


train_input = f's3://{s3_bucket}/{prefix}/srt_train.csv'
print(train_input)

In [ ]:
sklearn_preprocessor.fit({'train': train_input})

## Batch transform the training data 
With the preprocessor fitted, we can now preprocess our training data. We'll use **batch transform**, storing the output right back into s3.

In [ ]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(instance_count=1,
                                               instance_type='ml.m4.xlarge',
                                               assemble_with='Line',
                                               accept='text/csv')

In [ ]:
# Preprocess training input
transformer.transform(train_input, content_type='text/csv')
print(f'Waiting for transform job: {transformer.latest_transform_job.job_name}')
transformer.wait()
preprocessed_train = transformer.output_path

# Fit a LinearLearner
Here we'll use our preprocessed training data to fit a LinearLearner Model.

In [ ]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
ll_image = get_image_uri(boto3.Session().region_name, 'linear-learner')

In [ ]:
s3_ll_output_key_prefix = "ll_training_output"
s3_ll_output_location = 's3://{}/{}/{}/{}'.format(s3_bucket, prefix, s3_ll_output_key_prefix, 'll_model')

ll_estimator = sagemaker.estimator.Estimator(ll_image,
                                             role, 
                                             train_instance_count=1, 
                                             train_instance_type='ml.m4.2xlarge',
                                             train_volume_size = 20,
                                             train_max_run = 3600,
                                             input_mode= 'File',
                                             output_path=s3_ll_output_location,
                                             sagemaker_session=sagemaker_session)

ll_estimator.set_hyperparameters(feature_dim=10, predictor_type='regressor', mini_batch_size=32)

ll_train_data = sagemaker.session.s3_input(preprocessed_train, 
                                           distribution='FullyReplicated',
                                           content_type='text/csv', 
                                           s3_data_type='S3Prefix')

data_channels = {'train': ll_train_data}

ll_estimator.fit(inputs=data_channels, logs=True)

# Set up the inference pipeline

We can use the Pipeline Model here. This sets up a list of models in a single endpoint; in this example, we configure our pipeline model with the fitted Scikit-learn inference model and the fitted Linear Learner model.

In [ ]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
linear_learner_model = ll_estimator.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix
sm_model = PipelineModel(name=model_name, 
                         role=role, 
                         models=[scikit_learn_inferencee_model, linear_learner_model])

sm_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

# Make a request to the pipeline endpoint 
Here we'll use the deployed model to get a prediction for some dummy data. 

Below, the `content_type` field configures the first container, while the `accept` field configures the last container.

We make our request with the payload in `text/csv` format, since that is what our script currently supports. If other formats need to be supported, this would have to be added to the `output_fn()` method in our entrypoint. Note that we set the `accpet` to `application/json`, since LinearLearner does not support `text/csv`

In [ ]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

payload = 'This document is not compliant!'
y_true = 0
predictor = RealTimePredictor(endpoint=endpoint_name,
                              sagemaker_session=sagemaker_session,
                              serializer=csv_serializer,
                              content_type=CONTENT_TYPE_CSV,
                              accept=CONTENT_TYPE_JSON)

print(predictor.predict(payload))

# Delete Endpoint 
Once we are finished with the endpoint, we clean up the resources.

In [ ]:
sm_client = sagemaker_session.boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)